In [1185]:
import pandas as  pd 
import numpy as np 
import matplotlib.pyplot as plt 
from  datetime import datetime

In [1186]:
# load data 
train=pd.read_csv("C:/Users/G I E/Desktop/hack for earth/train.csv")
sample_sub=pd.read_csv("C:/Users/G I E/Desktop/hack for earth/Sample_sub.csv")

In [1187]:
sample_sub.head()

,ID,Capture_Number
0,CaptureSite_0_201901,7
1,CaptureSite_0_201902,1
2,CaptureSite_0_201903,5
3,CaptureSite_0_201904,2
4,CaptureSite_0_201905,3


In [1188]:
train.head()

,Rescue_ID,Date_TimeCaught,Researcher,CaptureSite,ForagingGround,CaptureMethod,Fisher,LandingSite,Species,Tag_1,...,Lost_Tags,T_Number,CCL_cm,CCW_cm,Weight_Kg,Sex,TurtleCharacteristics,Status,ReleaseSite,Date_TimeRelease
0,2000_RE_0060,2000-12-22,Researcher_25,CaptureSite_0,Ocean,Net,Fisher_1072,LandingSite_CaptureSiteCategory_2,Species_6,CC00147,...,NaN,NaN,64.70,62.60,NaN,Unknown,algae at rear of shell,Released,ReleaseSite_50,22/12/00
1,2001_RE_0187,2001-10-28,Researcher_6,CaptureSite_0,Ocean,Net,Fisher_520,LandingSite_CaptureSiteCategory_2,Species_6,W442,...,NaN,NaN,35.85,31.35,NaN,Unknown,multiple b's on front flippers& a lot of alga...,Released,ReleaseSite_62,28/10/01
2,2001_RE_0197,2001-11-01,Researcher_6,CaptureSite_0,Ocean,Net,Fisher_1669,LandingSite_CaptureSiteCategory_2,Species_5,KE0376,...,NaN,NaN,51.80,49.20,NaN,Unknown,clean,Released,ReleaseSite_50,01/11/01
3,2002_RE_0031,2002-03-11,Researcher_32,CaptureSite_0,Ocean,Net,Fisher_1798,LandingSite_CaptureSiteCategory_2,Species_6,CC00302,...,NaN,NaN,60.50,59.00,NaN,Unknown,1 b 3 CS+ calcerous algae at rear end of shell...,Released,ReleaseSite_50,11/03/02
4,2002_RE_0118,2002-08-08,Researcher_25,CaptureSite_0,Ocean,Beached,Fisher_1918,LandingSite_CaptureSiteCategory_2,Species_5,NotTagged_0113,...,NaN,NaN,34.70,33.00,NaN,Unknown,very lively+ right eye is hanging out + swolle...,Released,ReleaseSite_62,08/08/02


In [1189]:
# split Id to Year woy and CaptureSite
sample_sub["year_woy"]=(sample_sub.ID.apply(lambda x: x.split("_")[-1])).astype(int)
sample_sub["CaptureSite"]=sample_sub.ID.apply(lambda x: ("_").join(x.split("_")[0:-1]))

In [1190]:
# Create Time features from Date_TimeCaught 
train["Date_TimeCaught"]=pd.to_datetime(train["Date_TimeCaught"])
print(train.Date_TimeCaught.min(),train.Date_TimeCaught.max())
train["year"]=train.Date_TimeCaught.dt.year
train["week_of_year"]=train.Date_TimeCaught.dt.weekofyear
train["year_woy"]=train.year*100+train.week_of_year

1998-04-17 00:00:00 2018-12-31 00:00:00


In [1191]:
keys=pd.concat([train[["year_woy","CaptureSite"]],sample_sub[["year_woy","CaptureSite"]]])
# keys.sort_values(["CaptureSite","year_woy"],inplace=True)
CaptureSite_min_year_woy=keys.groupby("CaptureSite").year_woy.min().rename("year_woy").reset_index()
CaptureSite_min_year_woy.head()

,CaptureSite,year_woy
0,CaptureSite_0,200051
1,CaptureSite_1,200123
2,CaptureSite_10,200050
3,CaptureSite_11,199828
4,CaptureSite_12,200107


In [1192]:
range_year_woy=pd.DataFrame()
range_year_woy["Date_TimeCaught"]=pd.date_range(start=train.Date_TimeCaught.min(),end=datetime(2019,10,31))
range_year_woy["year"]=range_year_woy.Date_TimeCaught.dt.year
range_year_woy["week_of_year"]=range_year_woy.Date_TimeCaught.dt.weekofyear
range_year_woy["year_woy"]=range_year_woy.year*100+range_year_woy.week_of_year
range_year_woy.drop_duplicates(["year_woy"],inplace=True)

In [1193]:
final_data=[]
for site , year_woy in zip(CaptureSite_min_year_woy.CaptureSite.values,CaptureSite_min_year_woy.year_woy.values) :

    one_site_df=range_year_woy[range_year_woy.year_woy>=year_woy]
    one_site_df["CaptureSite"]=site
    final_data.append(one_site_df)
final_data=pd.concat(final_data)
len(final_data)

C:\Users\G I E\Anaconda\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


28048

In [1194]:
range_year_woy

,Date_TimeCaught,year,week_of_year,year_woy
0,1998-04-17,1998,16,199816
3,1998-04-20,1998,17,199817
10,1998-04-27,1998,18,199818
17,1998-05-04,1998,19,199819
24,1998-05-11,1998,20,199820
31,1998-05-18,1998,21,199821
38,1998-05-25,1998,22,199822
45,1998-06-01,1998,23,199823
52,1998-06-08,1998,24,199824
59,1998-06-15,1998,25,199825


In [1195]:
Target=train.groupby(["year_woy","CaptureSite"]).CaptureSite.count().rename("Capture_Number").reset_index()
final_data=final_data.merge(Target,on=["year_woy","CaptureSite"],how="left")
final_data.Capture_Number.fillna(0,inplace=True)

In [1196]:
final_data.Capture_Number.fillna(0).value_counts(True)

0.0     0.716308
1.0     0.147461
2.0     0.059469
3.0     0.030626
4.0     0.015759
5.0     0.009199
6.0     0.005990
7.0     0.005455
8.0     0.002603
9.0     0.002104
10.0    0.001604
11.0    0.000998
12.0    0.000606
14.0    0.000535
13.0    0.000499
15.0    0.000214
16.0    0.000178
17.0    0.000107
28.0    0.000036
22.0    0.000036
19.0    0.000036
35.0    0.000036
20.0    0.000036
21.0    0.000036
18.0    0.000036
23.0    0.000036
Name: Capture_Number, dtype: float64

In [1197]:
#final_data=final_data[(final_data.year.isin([]))]

In [1198]:
final_data.head()

,Date_TimeCaught,year,week_of_year,year_woy,CaptureSite,Capture_Number
0,2000-01-01,2000,52,200052,CaptureSite_0,0.0
1,2000-12-18,2000,51,200051,CaptureSite_0,1.0
2,2001-01-01,2001,1,200101,CaptureSite_0,0.0
3,2001-01-08,2001,2,200102,CaptureSite_0,0.0
4,2001-01-15,2001,3,200103,CaptureSite_0,0.0


In [1199]:
final_data["catching_day"]=final_data.Date_TimeCaught.dt.day

In [1200]:
final_data["catching_month"]=final_data.Date_TimeCaught.dt.month

In [1201]:
final_data["day_of_week"]=final_data.Date_TimeCaught.dt.weekday

In [1202]:
final_data['saison']=np.zeros(final_data.shape[0])
for i in range(train.shape[0]):
    if final_data['catching_month'][i] in ['01','02','12']:
        final_data['saison']= 'hiver'
    elif final_data['catching_month'][i] in ['03','04','05']:
        final_data['saison']='printemps'
    elif final_data['catching_month'][i] in ['06','07','08']:
        final_data['saison']='ete'
    else:
        final_data['saison']='automne'

In [1203]:
df=pd.read_csv("C:/Users/G I E/Desktop/hack for earth/train.csv")

In [1204]:
df.head()

,Rescue_ID,Date_TimeCaught,Researcher,CaptureSite,ForagingGround,CaptureMethod,Fisher,LandingSite,Species,Tag_1,...,Lost_Tags,T_Number,CCL_cm,CCW_cm,Weight_Kg,Sex,TurtleCharacteristics,Status,ReleaseSite,Date_TimeRelease
0,2000_RE_0060,2000-12-22,Researcher_25,CaptureSite_0,Ocean,Net,Fisher_1072,LandingSite_CaptureSiteCategory_2,Species_6,CC00147,...,NaN,NaN,64.70,62.60,NaN,Unknown,algae at rear of shell,Released,ReleaseSite_50,22/12/00
1,2001_RE_0187,2001-10-28,Researcher_6,CaptureSite_0,Ocean,Net,Fisher_520,LandingSite_CaptureSiteCategory_2,Species_6,W442,...,NaN,NaN,35.85,31.35,NaN,Unknown,multiple b's on front flippers& a lot of alga...,Released,ReleaseSite_62,28/10/01
2,2001_RE_0197,2001-11-01,Researcher_6,CaptureSite_0,Ocean,Net,Fisher_1669,LandingSite_CaptureSiteCategory_2,Species_5,KE0376,...,NaN,NaN,51.80,49.20,NaN,Unknown,clean,Released,ReleaseSite_50,01/11/01
3,2002_RE_0031,2002-03-11,Researcher_32,CaptureSite_0,Ocean,Net,Fisher_1798,LandingSite_CaptureSiteCategory_2,Species_6,CC00302,...,NaN,NaN,60.50,59.00,NaN,Unknown,1 b 3 CS+ calcerous algae at rear end of shell...,Released,ReleaseSite_50,11/03/02
4,2002_RE_0118,2002-08-08,Researcher_25,CaptureSite_0,Ocean,Beached,Fisher_1918,LandingSite_CaptureSiteCategory_2,Species_5,NotTagged_0113,...,NaN,NaN,34.70,33.00,NaN,Unknown,very lively+ right eye is hanging out + swolle...,Released,ReleaseSite_62,08/08/02


In [1205]:
a = pd.DataFrame(df.groupby('CaptureSite').agg({'Researcher':['count']}))

In [1206]:
b = pd.DataFrame(df.groupby('CaptureSite').agg({'Fisher':['count']}))

In [1207]:
b.head()

,Fisher
,count
CaptureSite,
CaptureSite_0,95
CaptureSite_1,793
CaptureSite_10,464
CaptureSite_11,312
CaptureSite_12,573


In [1208]:
a.head()

,Researcher
,count
CaptureSite,
CaptureSite_0,95
CaptureSite_1,793
CaptureSite_10,464
CaptureSite_11,312
CaptureSite_12,573


In [1209]:
final_data = final_data.merge(a,on='CaptureSite')

In [1210]:
final_data = final_data.merge(b,on='CaptureSite')

In [1211]:
final_data.head()

,Date_TimeCaught,year,week_of_year,year_woy,CaptureSite,Capture_Number,catching_day,catching_month,day_of_week,saison,"(Researcher, count)","(Fisher, count)"
0,2000-01-01,2000,52,200052,CaptureSite_0,0.0,1,1,5,automne,95,95
1,2000-12-18,2000,51,200051,CaptureSite_0,1.0,18,12,0,automne,95,95
2,2001-01-01,2001,1,200101,CaptureSite_0,0.0,1,1,0,automne,95,95
3,2001-01-08,2001,2,200102,CaptureSite_0,0.0,8,1,0,automne,95,95
4,2001-01-15,2001,3,200103,CaptureSite_0,0.0,15,1,0,automne,95,95


In [1212]:
final_data.head()

,Date_TimeCaught,year,week_of_year,year_woy,CaptureSite,Capture_Number,catching_day,catching_month,day_of_week,saison,"(Researcher, count)","(Fisher, count)"
0,2000-01-01,2000,52,200052,CaptureSite_0,0.0,1,1,5,automne,95,95
1,2000-12-18,2000,51,200051,CaptureSite_0,1.0,18,12,0,automne,95,95
2,2001-01-01,2001,1,200101,CaptureSite_0,0.0,1,1,0,automne,95,95
3,2001-01-08,2001,2,200102,CaptureSite_0,0.0,8,1,0,automne,95,95
4,2001-01-15,2001,3,200103,CaptureSite_0,0.0,15,1,0,automne,95,95


In [1213]:
category.head()

,CaptureSite,CaptureSiteCategory,Type
0,CaptureSite_0,CaptureSiteCategory_2,Type_1
1,CaptureSite_1,CaptureSiteCategory_2,Type_1
2,CaptureSite_10,CaptureSiteCategory_2,Type_1
3,CaptureSite_11,CaptureSiteCategory_0,Type_0
4,CaptureSite_12,CaptureSiteCategory_2,Type_1


In [1214]:
category=pd.read_csv('C:/Users/G I E/Desktop/hack for earth/CaptureSite_category.csv')

In [1215]:
c = pd.DataFrame(category.groupby('CaptureSite').agg({'CaptureSiteCategory':['count']}))
d = pd.DataFrame(category.groupby('CaptureSite').agg({'Type':['count']}))

In [1216]:
c.head()

,CaptureSiteCategory
,count
CaptureSite,
CaptureSite_0,1
CaptureSite_1,1
CaptureSite_10,1
CaptureSite_11,1
CaptureSite_12,1


In [1217]:
final_data = final_data.merge(category,on='CaptureSite')

In [1218]:
final_data = final_data.merge(c,on='CaptureSite')

In [1219]:
final_data = final_data.merge(d,on='CaptureSite')

In [1220]:
del final_data['Date_TimeCaught']

In [1221]:
e = pd.DataFrame(final_data.groupby('saison').agg({'Capture_Number':['count']}))

In [1222]:
final_data = final_data.merge(e,on='saison')

In [1223]:
final_data.head()

,year,week_of_year,year_woy,CaptureSite,Capture_Number,catching_day,catching_month,day_of_week,saison,"(Researcher, count)","(Fisher, count)",CaptureSiteCategory,Type,"(CaptureSiteCategory, count)","(Type, count)","(Capture_Number, count)"
0,2000,52,200052,CaptureSite_0,0.0,1,1,5,automne,95,95,CaptureSiteCategory_2,Type_1,1,1,28048
1,2000,51,200051,CaptureSite_0,1.0,18,12,0,automne,95,95,CaptureSiteCategory_2,Type_1,1,1,28048
2,2001,1,200101,CaptureSite_0,0.0,1,1,0,automne,95,95,CaptureSiteCategory_2,Type_1,1,1,28048
3,2001,2,200102,CaptureSite_0,0.0,8,1,0,automne,95,95,CaptureSiteCategory_2,Type_1,1,1,28048
4,2001,3,200103,CaptureSite_0,0.0,15,1,0,automne,95,95,CaptureSiteCategory_2,Type_1,1,1,28048


In [1224]:
#f = pd.DataFrame(final_data.groupby('week_of_year').agg({'Capture_Number':['count']}))
#f.head()

In [1225]:
#final_data['probabilities'] = final_data['(Capture_Number,count)']/7957

In [1226]:
#final_data = final_data.merge(f,on='week_of_year')

In [1227]:
#f = pd.DataFrame(final_data.groupby('week_of_year').agg({'CaptureSite':['count']}))

In [1228]:
#final_data = final_data.merge(f,on='week_of_year')

In [1229]:
int(final_data.CaptureSite.values[-1].split('_')[1])

9

In [1230]:
for i in range(len(final_data)):
    final_data.loc[i, 'capturesite_number'] = int(final_data.CaptureSite.values[i].split('_')[1])

In [1231]:
final_data['CaptureSite_FE'] = final_data['CaptureSite'].map(final_data['CaptureSite'].value_counts(True))

In [1232]:
final_data

,year,week_of_year,year_woy,CaptureSite,Capture_Number,catching_day,catching_month,day_of_week,saison,"(Researcher, count)","(Fisher, count)",CaptureSiteCategory,Type,"(CaptureSiteCategory, count)","(Type, count)","(Capture_Number, count)",capturesite_number,CaptureSite_FE
0,2000,52,200052,CaptureSite_0,0.0,1,1,5,automne,95,95,CaptureSiteCategory_2,Type_1,1,1,28048,0.0,0.035225
1,2000,51,200051,CaptureSite_0,1.0,18,12,0,automne,95,95,CaptureSiteCategory_2,Type_1,1,1,28048,0.0,0.035225
2,2001,1,200101,CaptureSite_0,0.0,1,1,0,automne,95,95,CaptureSiteCategory_2,Type_1,1,1,28048,0.0,0.035225
3,2001,2,200102,CaptureSite_0,0.0,8,1,0,automne,95,95,CaptureSiteCategory_2,Type_1,1,1,28048,0.0,0.035225
4,2001,3,200103,CaptureSite_0,0.0,15,1,0,automne,95,95,CaptureSiteCategory_2,Type_1,1,1,28048,0.0,0.035225
5,2001,4,200104,CaptureSite_0,0.0,22,1,0,automne,95,95,CaptureSiteCategory_2,Type_1,1,1,28048,0.0,0.035225
6,2001,5,200105,CaptureSite_0,0.0,29,1,0,automne,95,95,CaptureSiteCategory_2,Type_1,1,1,28048,0.0,0.035225
7,2001,6,200106,CaptureSite_0,0.0,5,2,0,automne,95,95,CaptureSiteCategory_2,Type_1,1,1,28048,0.0,0.035225
8,2001,7,200107,CaptureSite_0,0.0,12,2,0,automne,95,95,CaptureSiteCategory_2,Type_1,1,1,28048,0.0,0.035225
9,2001,8,200108,CaptureSite_0,0.0,19,2,0,automne,95,95,CaptureSiteCategory_2,Type_1,1,1,28048,0.0,0.035225


In [1233]:
pd.get_dummies(final_data,columns=['week_of_year'])

,year,year_woy,CaptureSite,Capture_Number,catching_day,catching_month,day_of_week,saison,"(Researcher, count)","(Fisher, count)",...,week_of_year_44,week_of_year_45,week_of_year_46,week_of_year_47,week_of_year_48,week_of_year_49,week_of_year_50,week_of_year_51,week_of_year_52,week_of_year_53
0,2000,200052,CaptureSite_0,0.0,1,1,5,automne,95,95,...,0,0,0,0,0,0,0,0,1,0
1,2000,200051,CaptureSite_0,1.0,18,12,0,automne,95,95,...,0,0,0,0,0,0,0,1,0,0
2,2001,200101,CaptureSite_0,0.0,1,1,0,automne,95,95,...,0,0,0,0,0,0,0,0,0,0
3,2001,200102,CaptureSite_0,0.0,8,1,0,automne,95,95,...,0,0,0,0,0,0,0,0,0,0
4,2001,200103,CaptureSite_0,0.0,15,1,0,automne,95,95,...,0,0,0,0,0,0,0,0,0,0
5,2001,200104,CaptureSite_0,0.0,22,1,0,automne,95,95,...,0,0,0,0,0,0,0,0,0,0
6,2001,200105,CaptureSite_0,0.0,29,1,0,automne,95,95,...,0,0,0,0,0,0,0,0,0,0
7,2001,200106,CaptureSite_0,0.0,5,2,0,automne,95,95,...,0,0,0,0,0,0,0,0,0,0
8,2001,200107,CaptureSite_0,0.0,12,2,0,automne,95,95,...,0,0,0,0,0,0,0,0,0,0
9,2001,200108,CaptureSite_0,0.0,19,2,0,automne,95,95,...,0,0,0,0,0,0,0,0,0,0


In [1234]:
pd.get_dummies(final_data,columns=['CaptureSite'])

,year,week_of_year,year_woy,Capture_Number,catching_day,catching_month,day_of_week,saison,"(Researcher, count)","(Fisher, count)",...,CaptureSite_CaptureSite_26,CaptureSite_CaptureSite_27,CaptureSite_CaptureSite_28,CaptureSite_CaptureSite_3,CaptureSite_CaptureSite_4,CaptureSite_CaptureSite_5,CaptureSite_CaptureSite_6,CaptureSite_CaptureSite_7,CaptureSite_CaptureSite_8,CaptureSite_CaptureSite_9
0,2000,52,200052,0.0,1,1,5,automne,95,95,...,0,0,0,0,0,0,0,0,0,0
1,2000,51,200051,1.0,18,12,0,automne,95,95,...,0,0,0,0,0,0,0,0,0,0
2,2001,1,200101,0.0,1,1,0,automne,95,95,...,0,0,0,0,0,0,0,0,0,0
3,2001,2,200102,0.0,8,1,0,automne,95,95,...,0,0,0,0,0,0,0,0,0,0
4,2001,3,200103,0.0,15,1,0,automne,95,95,...,0,0,0,0,0,0,0,0,0,0
5,2001,4,200104,0.0,22,1,0,automne,95,95,...,0,0,0,0,0,0,0,0,0,0
6,2001,5,200105,0.0,29,1,0,automne,95,95,...,0,0,0,0,0,0,0,0,0,0
7,2001,6,200106,0.0,5,2,0,automne,95,95,...,0,0,0,0,0,0,0,0,0,0
8,2001,7,200107,0.0,12,2,0,automne,95,95,...,0,0,0,0,0,0,0,0,0,0
9,2001,8,200108,0.0,19,2,0,automne,95,95,...,0,0,0,0,0,0,0,0,0,0


In [1235]:
pd.get_dummies(final_data,columns=['saison'])

,year,week_of_year,year_woy,CaptureSite,Capture_Number,catching_day,catching_month,day_of_week,"(Researcher, count)","(Fisher, count)",CaptureSiteCategory,Type,"(CaptureSiteCategory, count)","(Type, count)","(Capture_Number, count)",capturesite_number,CaptureSite_FE,saison_automne
0,2000,52,200052,CaptureSite_0,0.0,1,1,5,95,95,CaptureSiteCategory_2,Type_1,1,1,28048,0.0,0.035225,1
1,2000,51,200051,CaptureSite_0,1.0,18,12,0,95,95,CaptureSiteCategory_2,Type_1,1,1,28048,0.0,0.035225,1
2,2001,1,200101,CaptureSite_0,0.0,1,1,0,95,95,CaptureSiteCategory_2,Type_1,1,1,28048,0.0,0.035225,1
3,2001,2,200102,CaptureSite_0,0.0,8,1,0,95,95,CaptureSiteCategory_2,Type_1,1,1,28048,0.0,0.035225,1
4,2001,3,200103,CaptureSite_0,0.0,15,1,0,95,95,CaptureSiteCategory_2,Type_1,1,1,28048,0.0,0.035225,1
5,2001,4,200104,CaptureSite_0,0.0,22,1,0,95,95,CaptureSiteCategory_2,Type_1,1,1,28048,0.0,0.035225,1
6,2001,5,200105,CaptureSite_0,0.0,29,1,0,95,95,CaptureSiteCategory_2,Type_1,1,1,28048,0.0,0.035225,1
7,2001,6,200106,CaptureSite_0,0.0,5,2,0,95,95,CaptureSiteCategory_2,Type_1,1,1,28048,0.0,0.035225,1
8,2001,7,200107,CaptureSite_0,0.0,12,2,0,95,95,CaptureSiteCategory_2,Type_1,1,1,28048,0.0,0.035225,1
9,2001,8,200108,CaptureSite_0,0.0,19,2,0,95,95,CaptureSiteCategory_2,Type_1,1,1,28048,0.0,0.035225,1


In [1236]:
from sklearn import preprocessing
for f in final_data.columns:
    if final_data[f].dtype=='object'  : 
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(final_data[f].values))  
        final_data[f] = lbl.transform(list(final_data[f].values)) 

In [1237]:
train=final_data[final_data.year<2019]


In [1238]:
test=final_data[final_data.year==2019]

In [1239]:
del test['Capture_Number']

In [1240]:
y_train=train['Capture_Number']
X_train=train.drop('Capture_Number',axis=1)

In [1241]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.33, random_state=18)


lgb_train = lgb.Dataset(data=x_train, label=y_train)


lgb_eval = lgb.Dataset(data=x_val, label=y_val)

In [1242]:
final_data.head()

,year,week_of_year,year_woy,CaptureSite,Capture_Number,catching_day,catching_month,day_of_week,saison,"(Researcher, count)","(Fisher, count)",CaptureSiteCategory,Type,"(CaptureSiteCategory, count)","(Type, count)","(Capture_Number, count)",capturesite_number,CaptureSite_FE
0,2000,52,200052,0,0.0,1,1,5,0,95,95,2,1,1,1,28048,0.0,0.035225
1,2000,51,200051,0,1.0,18,12,0,0,95,95,2,1,1,1,28048,0.0,0.035225
2,2001,1,200101,0,0.0,1,1,0,0,95,95,2,1,1,1,28048,0.0,0.035225
3,2001,2,200102,0,0.0,8,1,0,0,95,95,2,1,1,1,28048,0.0,0.035225
4,2001,3,200103,0,0.0,15,1,0,0,95,95,2,1,1,1,28048,0.0,0.035225


In [1447]:
params = {'task': 'train', 'boosting_type': 'gbdt', 'objective': 'regression', 'metric': 'rmse', 
          'learning_rate': 0.024, 'num_leaves': 80, 'verbose': 0 ,
          'reg_alpha':0.4, 'reg_lambda':0.4, 'max_depth':-1,'lambda':0.03,
          'feature_fraction':0.4,'bagging_fraction':0.4, 'min_data_in_leaf':120,'n_estimators':1000}
model1 = lgb.train(params, lgb_train, valid_sets=lgb_eval, early_stopping_rounds=80, verbose_eval=80)

Training until validation scores don't improve for 80 rounds.
[80]	valid_0's rmse: 1.2541
[160]	valid_0's rmse: 1.21773
[240]	valid_0's rmse: 1.20538
[320]	valid_0's rmse: 1.19955
[400]	valid_0's rmse: 1.19598
[480]	valid_0's rmse: 1.19497
[560]	valid_0's rmse: 1.19369
[640]	valid_0's rmse: 1.19339
[720]	valid_0's rmse: 1.19354
Early stopping, best iteration is:
[675]	valid_0's rmse: 1.19304


In [1426]:
from xgboost import XGBRegressor

In [1438]:
model2 = XGBRegressor(
    max_depth=2,
    learning_rate=0.07,
    n_estimators=1000,
    min_child_weight=300, 
    colsample_bytree=1, 
    subsample=0.8, 
    eta=0.3,    
    seed=42,
gamma=10
)

model2.fit(
    x_train, 
    y_train, 
    eval_metric="rmse", 
    eval_set=[(x_train, y_train), (x_val, y_val)], 
    verbose=True, 
    early_stopping_rounds = 10)


[19:23:20] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-rmse:1.55349	validation_1-rmse:1.56255
Multiple eval metrics have been passed: 'validation_1-rmse' will be used for early stopping.

Will train until validation_1-rmse hasn't improved in 10 rounds.
[1]	validation_0-rmse:1.52752	validation_1-rmse:1.53724
[2]	validation_0-rmse:1.50397	validation_1-rmse:1.51513
[3]	validation_0-rmse:1.48344	validation_1-rmse:1.49615
[4]	validation_0-rmse:1.46485	validation_1-rmse:1.47865
[5]	validation_0-rmse:1.4487	validation_1-rmse:1.46357
[6]	validation_0-rmse:1.4347	validation_1-rmse:1.45002
[7]	validation_0-rmse:1.42216	validation_1-rmse:1.43783
[8]	validation_0-rmse:1.4108	validation_1-rmse:1.42694
[9]	validation_0-rmse:1.40087	validation_1-rmse:1.41778
[10]	validation_0-rmse:1.39226	validation_1-rmse:1.40935
[11]	validation_0-rmse:1.38229	validation_1-rmse:1.40008
[12]	v

[139]	validation_0-rmse:1.2744	validation_1-rmse:1.29705
[140]	validation_0-rmse:1.27411	validation_1-rmse:1.29693
[141]	validation_0-rmse:1.27405	validation_1-rmse:1.29688
[142]	validation_0-rmse:1.27394	validation_1-rmse:1.29683
[143]	validation_0-rmse:1.27362	validation_1-rmse:1.29647
[144]	validation_0-rmse:1.27353	validation_1-rmse:1.29642
[145]	validation_0-rmse:1.27346	validation_1-rmse:1.2964
[146]	validation_0-rmse:1.27342	validation_1-rmse:1.2963
[147]	validation_0-rmse:1.27336	validation_1-rmse:1.29632
[148]	validation_0-rmse:1.27331	validation_1-rmse:1.29629
[149]	validation_0-rmse:1.27331	validation_1-rmse:1.29629
[150]	validation_0-rmse:1.27327	validation_1-rmse:1.29622
[151]	validation_0-rmse:1.27327	validation_1-rmse:1.29622
[152]	validation_0-rmse:1.27247	validation_1-rmse:1.29501
[153]	validation_0-rmse:1.27247	validation_1-rmse:1.29501
[154]	validation_0-rmse:1.27247	validation_1-rmse:1.29501
[155]	validation_0-rmse:1.27221	validation_1-rmse:1.29488
[156]	validation_

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, eta=0.3, gamma=10,
       importance_type='gain', learning_rate=0.07, max_delta_step=0,
       max_depth=2, min_child_weight=300, missing=None, n_estimators=1000,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=42, silent=None,
       subsample=0.8, verbosity=1)

In [1439]:
#from catboost import CatBoostRegressor
#model3 = CatBoostRegressor(iterations=2000, learning_rate=0.05, depth=5)
# Fit model
#model3.fit(x_train,y_train)
# Get predictions


In [1440]:
test_preds1= model.predict(test)
test_preds2= model2.predict(test)
#test_preds3= model3.predict(test)
final_predictions = (test_preds1 + test_preds2)/2

In [1441]:
#stacked_predictions=np.column_stack((preds1,preds2))
#stacked_test_predictions=np.column_stack((test_preds1,test_preds2))


In [1442]:

#lgb_train = lgb.Dataset(data=stacked_predictions, label=y_val)


In [1443]:
#meta_model= lgb.train(params,lgb_train)

In [1444]:
#final_predictions = meta_model.predict(stacked_test_predictions)
sample_sub['Capture_Number']=final_predictions

In [1445]:
from IPython.display import FileLink
def create_submission(submission_file, submission_name):
    submission_file.to_csv(submission_name+".csv" , index=False)
    return FileLink(submission_name+".csv")

In [1446]:
create_submission(sample_sub, 'hfe16')

C:\Users\G I E\Downloads\hfe16.csv